# This notebook implements the entropy estimation of section 5.A in 
# https://arxiv.org/abs/2012.11197# 

In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import multinomial, geom, boltzmann, dlaplace
from scipy.stats import poisson
import os
import keras
from keras.models import Model
from keras.layers import Dense, Input, TimeDistributed, LSTM
from keras import optimizers
import tensorflow as tf

#  Simulated data generators 

In [ ]:
def zipf_dist(alpha, N, size):
    x = np.arange(1, N+1, dtype='float')
    weights = x ** (-alpha)
    weights /= weights.sum()
    bounded_zipf = stats.rv_discrete(name='bounded_zipf', values=(x, weights))
    data = np.reshape(np.array(bounded_zipf.rvs(size=size)), [-1, 1])
    return data, weights

In [ ]:
def zipf_dist(alpha, N, size):
    x = np.arange(1, N+1, dtype='float')
    weights = x ** (-alpha)
    weights /= weights.sum()
    bounded_zipf = stats.rv_discrete(name='bounded_zipf', values=(x, weights))
    data = np.reshape(np.array(bounded_zipf.rvs(size=size)), [-1, 1])
    return data, weights

# Estimating H(X)


In [54]:
def harmonic(n, alpha):
    a = 0
    for i in range(1, n):
        a += 1/i**alpha
    return a

In [55]:
def zipf_entropy(alphabet, alpha):
    p = np.arange(1, alphabet, dtype='float')**(-alpha)
    c = harmonic(alphabet, alpha)
    H_zipf = -(1/c)*np.sum(p*np.log(p/c))
    return H_zipf

In [56]:
def geom_entropy(weights):
    return -np.sum(weights*np.log(weights))

In [5]:
# Other utils
def convert_to_one_hot(y, dict_size=None):
    if dict_size is None:
        dict_size = np.unique(y).shape[0]
    y_hot = np.eye(dict_size)[y.astype('int32')]
    return y_hot

def make_one_hot(y, dims, dict_size=None):
    y_hot = []
    for i in range(dims):
        y_hot.append(convert_to_one_hot(y[:, i], dict_size))
    return y_hot

def make_rnn_data(time_series, sequence_length):
    seq_lst = []
    time_series = np.array(time_series)
    for i in range((len(time_series)-sequence_length+1)):
        seq_lst.append(time_series[i:(i+sequence_length), :])
    return np.array(seq_lst)

#  model definition

In [7]:
def model_basic_classification(input_shape, class_size):
    l0 = Input(shape=input_shape, dtype = 'float32', name = 'input_l')
    X = LSTM(units=50, return_sequences=True, kernel_initializer='glorot_normal', name = 'l2')(l0)
    output = TimeDistributed(Dense(class_size, activation='softmax'))(X)
    model = Model(input = [l0], outputs =  output )
    return model

# Generating data

In [63]:
# Generate data from zipf, Geometric or mix 
source = 'zipf'
alpha = 1
alphabet = 10**5
size = 1000
p = 1/alphabet
if source == 'zipf':
    data, _ = zipf_dist(alpha, alphabet, size)
    H_true = zipf_entropy(alphabet, alpha)
elif source == 'geo':
    data, weights = geom_dist(p, alphabet, size)
    H_true = geom_entropy(weights)
    
# Convert symbols to binary representation
dims = 25 # size of the binary representation, chosen by the dectated by the alphabet size
vf = np.vectorize(np.binary_repr)
data = vf(data, width=dims)
lst = []
for i in data:
    lst.append([int(j) for j in i[0]])
data = np.array(lst)

# Prepare RNN input and output
data_shift = np.hstack([data[:, 1:], np.zeros(shape=(len(data), 1))])
data = np.expand_dims(data, axis=2)
seq_lst = []
for i in data_shift:
    y = np.reshape(i, (-1, 1))
    seq_lst.append(make_one_hot(y, 1, 2))
data_shift = np.array(seq_lst)
data_shift = np.squeeze(data_shift, axis=1)

# Estimation

In [64]:
seq_length = 25
epochs = 200
batch_size = 64
bins = 2

model = model_basic_classification([seq_length, 1], 2)
opt = optimizers.Adam()
model.compile(loss='categorical_crossentropy', optimizer=opt)
model.fit(data, data_shift, epochs=epochs, batch_size=batch_size, shuffle=True, verbose=0)
CE = model.predict(data)*data_shift
CE = np.mean(-np.log(CE.max(axis=2)).sum(axis=1))
y = data[:, 0]
p_1 = (np.sum(y)+10**-5)/float(size)
H_1 = -np.sum(np.array([p_1, 1-p_1] )*np.log([p_1, 1-p_1])) # plug in estimator
CE = CE + H_1

print('H estimated', CE)
print('H true', H_true)

C:\Users\shale\Anaconda3\envs\tf_1_15\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ti..., inputs=[<tf.Tenso...)`
  """


H estimated 8.014326745360242
H true 7.967998491428407
